In [6]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import sqlite3
import scipy.stats

# Question 1)

In [22]:
# Connection to the db and fetch all the data via pandas function to put in a dataframe
conn = sqlite3.connect(r"C:\Users\ranch\Desktop\Skema\S2\Python 2\projectss_db.db")
rawData = pd.read_sql_query("SELECT * FROM histo", conn)
rawData.info()
rawData.head(5)

rawData['DATE'] =  pd.to_datetime(rawData['DATE'], format='%Y-%m-%d')
rawData = rawData.set_index('DATE')
rawData.drop(['ID'], axis=1, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9961 entries, 0 to 9960
Data columns (total 5 columns):
ID        9961 non-null int64
DATE      9961 non-null object
TICKER    9961 non-null object
FIELD     9961 non-null object
VALUE     9961 non-null float64
dtypes: float64(1), int64(1), object(3)
memory usage: 389.2+ KB


In [23]:
# Check the ticker used for each underlying
rawData['TICKER'].unique()

array(['SX5E Index', 'SPX Index', 'XAUUSD Curncy', 'CL1 Comdty'],
      dtype=object)

In [26]:
# Create a classe for each underlying
class Underlying():
    def __init__(self, ticker, rawData):
        self.df = rawData[rawData['TICKER'] == ticker]
        self.name = ticker
        self.log_ret = np.log(self.df['VALUE']) - np.log(self.df['VALUE'].shift(1))
        self.log_ret = self.log_ret[1:len(self.log_ret)]
        self.log_ret = self.log_ret.resample('MS').sum()
    
    def compute_mean(self):
        return self.log_ret.mean()
    
    def compute_variance(self):
        return np.var(self.log_ret)
    
    def compute_skew(self):
        return scipy.stats.skew(self.log_ret)
    
    def compute_kurtosis(self):
        return scipy.stats.kurtosis(self.log_ret)

In [27]:
# Create instance and compute value
sp = Underlying("SPX Index", rawData)
sx = Underlying("SX5E Index", rawData)
gold = Underlying("XAUUSD Curncy", rawData)
wti = Underlying("CL1 Comdty", rawData)
underlyings = [sp, sx, gold, wti]

for x in underlyings:
    print(x.name, x.compute_mean(), x.compute_variance(), x.compute_skew() ,x.compute_kurtosis())

SPX Index 0.0077046998086403255 0.0014029387726194103 -0.7708798162333944 1.6191163790500642
SX5E Index 0.00073180778785957 0.0022212810207719543 -0.7705677482261976 1.0960105261244246
XAUUSD Curncy 0.0020344019495397475 0.002109952972877925 -0.049585156194427056 0.057939095526308826
CL1 Comdty -0.011105887751121604 0.012542146593394761 -2.853685994077046 17.177048873814414
